# Type Objects

Perhaps one of the most important structures of the Python object system
is the structure that defines a new type: the :c`PyTypeObject`
structure. Type objects can be handled using any of the :c`PyObject_\*`
or :c`PyType_\*` functions, but do not offer much that's interesting to
most Python applications. These objects are fundamental to how objects
behave, so they are very important to the interpreter itself and to any
extension module that implements new types.

Type objects are fairly large compared to most of the standard types.
The reason for the size is that each type object stores a large number
of values, mostly C function pointers, each of which implements a small
part of the type's functionality. The fields of the type object are
examined in detail in this section. The fields will be described in the
order in which they occur in the structure.

In addition to the following quick reference, the `typedef-examples`
section provides at-a-glance insight into the meaning and use of
:c`PyTypeObject`.

## Quick Reference

### "tp slots"

### sub-slots

| Slot                                            | `Type <slot-typedefs-table>` | special methods                  |
|---------------------------|------------------|-------------|
| :c`~PyAsyncMethods.am_await`                    | :c`unaryfunc`                | \_\_await\_\_                    |
| :c`~PyAsyncMethods.am_aiter`                    | :c`unaryfunc`                | \_\_aiter\_\_                    |
| :c`~PyAsyncMethods.am_anext`                    | :c`unaryfunc`                | \_\_anext\_\_                    |
| :c`~PyAsyncMethods.am_send`                     | :c`sendfunc`                 |                                  |
|                                                 |                              |                                  |
| :c`~PyNumberMethods.nb_add`                     | :c`binaryfunc`               | \_\_add\_\_ \_\_radd\_\_         |
| :c`~PyNumberMethods.nb_inplace_add`             | :c`binaryfunc`               | \_\_iadd\_\_                     |
| :c`~PyNumberMethods.nb_subtract`                | :c`binaryfunc`               | \_\_sub\_\_ \_\_rsub\_\_         |
| :c`~PyNumberMethods.nb_inplace_subtract`        | :c`binaryfunc`               | \_\_isub\_\_                     |
| :c`~PyNumberMethods.nb_multiply`                | :c`binaryfunc`               | \_\_mul\_\_ \_\_rmul\_\_         |
| :c`~PyNumberMethods.nb_inplace_multiply`        | :c`binaryfunc`               | \_\_imul\_\_                     |
| :c`~PyNumberMethods.nb_remainder`               | :c`binaryfunc`               | \_\_mod\_\_ \_\_rmod\_\_         |
| :c`~PyNumberMethods.nb_inplace_remainder`       | :c`binaryfunc`               | \_\_imod\_\_                     |
| :c`~PyNumberMethods.nb_divmod`                  | :c`binaryfunc`               | \_\_divmod\_\_ \_\_rdivmod\_\_   |
| :c`~PyNumberMethods.nb_power`                   | :c`ternaryfunc`              | \_\_pow\_\_ \_\_rpow\_\_         |
| :c`~PyNumberMethods.nb_inplace_power`           | :c`ternaryfunc`              | \_\_ipow\_\_                     |
| :c`~PyNumberMethods.nb_negative`                | :c`unaryfunc`                | \_\_neg\_\_                      |
| :c`~PyNumberMethods.nb_positive`                | :c`unaryfunc`                | \_\_pos\_\_                      |
| :c`~PyNumberMethods.nb_absolute`                | :c`unaryfunc`                | \_\_abs\_\_                      |
| :c`~PyNumberMethods.nb_bool`                    | :c`inquiry`                  | \_\_bool\_\_                     |
| :c`~PyNumberMethods.nb_invert`                  | :c`unaryfunc`                | \_\_invert\_\_                   |
| :c`~PyNumberMethods.nb_lshift`                  | :c`binaryfunc`               | \_\_lshift\_\_ \_\_rlshift\_\_   |
| :c`~PyNumberMethods.nb_inplace_lshift`          | :c`binaryfunc`               | \_\_ilshift\_\_                  |
| :c`~PyNumberMethods.nb_rshift`                  | :c`binaryfunc`               | \_\_rshift\_\_ \_\_rrshift\_\_   |
| :c`~PyNumberMethods.nb_inplace_rshift`          | :c`binaryfunc`               | \_\_irshift\_\_                  |
| :c`~PyNumberMethods.nb_and`                     | :c`binaryfunc`               | \_\_and\_\_ \_\_rand\_\_         |
| :c`~PyNumberMethods.nb_inplace_and`             | :c`binaryfunc`               | \_\_iand\_\_                     |
| :c`~PyNumberMethods.nb_xor`                     | :c`binaryfunc`               | \_\_xor\_\_ \_\_rxor\_\_         |
| :c`~PyNumberMethods.nb_inplace_xor`             | :c`binaryfunc`               | \_\_ixor\_\_                     |
| :c`~PyNumberMethods.nb_or`                      | :c`binaryfunc`               | \_\_or\_\_ \_\_ror\_\_           |
| :c`~PyNumberMethods.nb_inplace_or`              | :c`binaryfunc`               | \_\_ior\_\_                      |
| :c`~PyNumberMethods.nb_int`                     | :c`unaryfunc`                | \_\_int\_\_                      |
| :c`~PyNumberMethods.nb_reserved`                | void \*                      |                                  |
| :c`~PyNumberMethods.nb_float`                   | :c`unaryfunc`                | \_\_float\_\_                    |
| :c`~PyNumberMethods.nb_floor_divide`            | :c`binaryfunc`               | \_\_floordiv\_\_                 |
| :c`~PyNumberMethods.nb_inplace_floor_divide`    | :c`binaryfunc`               | \_\_ifloordiv\_\_                |
| :c`~PyNumberMethods.nb_true_divide`             | :c`binaryfunc`               | \_\_truediv\_\_                  |
| :c`~PyNumberMethods.nb_inplace_true_divide`     | :c`binaryfunc`               | \_\_itruediv\_\_                 |
| :c`~PyNumberMethods.nb_index`                   | :c`unaryfunc`                | \_\_index\_\_                    |
| :c`~PyNumberMethods.nb_matrix_multiply`         | :c`binaryfunc`               | \_\_matmul\_\_ \_\_rmatmul\_\_   |
| :c`~PyNumberMethods.nb_inplace_matrix_multiply` | :c`binaryfunc`               | \_\_imatmul\_\_                  |
|                                                 |                              |                                  |
| :c`~PyMappingMethods.mp_length`                 | :c`lenfunc`                  | \_\_len\_\_                      |
| :c`~PyMappingMethods.mp_subscript`              | :c`binaryfunc`               | \_\_getitem\_\_                  |
| :c`~PyMappingMethods.mp_ass_subscript`          | :c`objobjargproc`            | \_\_setitem\_\_, \_\_delitem\_\_ |
|                                                 |                              |                                  |
| :c`~PySequenceMethods.sq_length`                | :c`lenfunc`                  | \_\_len\_\_                      |
| :c`~PySequenceMethods.sq_concat`                | :c`binaryfunc`               | \_\_add\_\_                      |
| :c`~PySequenceMethods.sq_repeat`                | :c`ssizeargfunc`             | \_\_mul\_\_                      |
| :c`~PySequenceMethods.sq_item`                  | :c`ssizeargfunc`             | \_\_getitem\_\_                  |
| :c`~PySequenceMethods.sq_ass_item`              | :c`ssizeobjargproc`          | \_\_setitem\_\_ \_\_delitem\_\_  |
| :c`~PySequenceMethods.sq_contains`              | :c`objobjproc`               | \_\_contains\_\_                 |
| :c`~PySequenceMethods.sq_inplace_concat`        | :c`binaryfunc`               | \_\_iadd\_\_                     |
| :c`~PySequenceMethods.sq_inplace_repeat`        | :c`ssizeargfunc`             | \_\_imul\_\_                     |
|                                                 |                              |                                  |
| :c`~PyBufferProcs.bf_getbuffer`                 | :c`getbufferproc`            |                                  |
| :c`~PyBufferProcs.bf_releasebuffer`             | :c`releasebufferproc`        |                                  |

### slot typedefs

<table>
<colgroup>
<col style="width: 36%" />
<col style="width: 36%" />
<col style="width: 27%" />
</colgroup>
<thead>
<tr class="header">
<th>typedef</th>
<th>Parameter Types</th>
<th>Return Type</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">allocfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyTypeObject</code> *<br />
:c<code class="interpreted-text"
role="type">Py_ssize_t</code></div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">destructor</code></td>
<td>void *</td>
<td>void</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">freefunc</code></td>
<td>void *</td>
<td>void</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">traverseproc</code></td>
<td><div class="line-block">void *<br />
:c<code class="interpreted-text" role="type">visitproc</code><br />
void *</div></td>
<td>int</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">newfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">initproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">reprfunc</code></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">getattrfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
const char *</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">setattrfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
const char *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">getattrofunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text"
role="type">setattrofunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">descrgetfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text"
role="type">descrsetfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">hashfunc</code></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
<td>Py_hash_t</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">richcmpfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
int</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">getiterfunc</code></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text"
role="type">iternextfunc</code></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">lenfunc</code></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
<td>:c<code class="interpreted-text" role="type">Py_ssize_t</code></td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text"
role="type">getbufferproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">Py_buffer</code> *<br />
int</div></td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">releasebufferproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">Py_buffer</code>
*</div></td>
<td>void</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">inquiry</code></td>
<td>void *</td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">unaryfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">binaryfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text" role="type">ternaryfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text"
role="type">ssizeargfunc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text"
role="type">Py_ssize_t</code></div></td>
<td>:c<code class="interpreted-text" role="type">PyObject</code> *</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">ssizeobjargproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text"
role="type">Py_ssize_t</code></div></td>
<td>int</td>
</tr>
<tr class="odd">
<td>:c<code class="interpreted-text" role="type">objobjproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
<tr class="even">
<td>:c<code class="interpreted-text"
role="type">objobjargproc</code></td>
<td><div class="line-block">:c<code class="interpreted-text"
role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code> *<br />
:c<code class="interpreted-text" role="type">PyObject</code>
*</div></td>
<td>int</td>
</tr>
</tbody>
</table>

See `slot-typedefs` below for more detail.

## PyTypeObject Definition

The structure definition for :c`PyTypeObject` can be found in
`Include/object.h`. For convenience of reference, this repeats the
definition found there:

../includes/typestruct.h

## PyObject Slots

The type object structure extends the :c`PyVarObject` structure. The
`ob_size` field is used for dynamic types (created by `type_new`,
usually called from a class statement). Note that :c`PyType_Type` (the
metatype) initializes :c`~PyTypeObject.tp_itemsize`, which means that
its instances (i.e. type objects) *must* have the `ob_size` field.

> This is the type object's reference count, initialized to `1` by the
> `PyObject_HEAD_INIT` macro. Note that for `statically allocated type
> objects <static-types>`, the type's instances (objects whose `ob_type`
> points back to the type) do *not* count as references. But for
> `dynamically allocated type objects <heap-types>`, the instances *do*
> count as references.
>
> **Inheritance:**
>
> This field is not inherited by subtypes.

> This is the type's type, in other words its metatype. It is
> initialized by the argument to the `PyObject_HEAD_INIT` macro, and its
> value should normally be `&PyType_Type`. However, for dynamically
> loadable extension modules that must be usable on Windows (at least),
> the compiler complains that this is not a valid initializer.
> Therefore, the convention is to pass `NULL` to the
> `PyObject_HEAD_INIT` macro and to initialize this field explicitly at
> the start of the module's initialization function, before doing
> anything else. This is typically done like this:
>
> ``` c
> Foo_Type.ob_type = &PyType_Type;
> ```
>
> This should be done before any instances of the type are created.
> :c`PyType_Ready` checks if `ob_type` is `NULL`, and if so, initializes
> it to the `ob_type` field of the base class. :c`PyType_Ready` will not
> change this field if it is non-zero.
>
> **Inheritance:**
>
> This field is inherited by subtypes.

## PyVarObject Slots

> For `statically allocated type objects <static-types>`, this should be
> initialized to zero. For `dynamically allocated type objects
> <heap-types>`, this field has a special internal meaning.
>
> **Inheritance:**
>
> This field is not inherited by subtypes.

## PyTypeObject Slots

Each slot has a section describing inheritance. If :c`PyType_Ready` may
set a value when the field is set to `NULL` then there will also be a
"Default" section. (Note that many fields set on :c`PyBaseObject_Type`
and :c`PyType_Type` effectively act as defaults.)

> Pointer to a NUL-terminated string containing the name of the type.
> For types that are accessible as module globals, the string should be
> the full module name, followed by a dot, followed by the type name;
> for built-in types, it should be just the type name. If the module is
> a submodule of a package, the full package name is part of the full
> module name. For example, a type named `T` defined in module `M` in
> subpackage `Q` in package `P` should have the
> :c`~PyTypeObject.tp_name` initializer `"P.Q.M.T"`.
>
> For `dynamically allocated type objects <heap-types>`, this should
> just be the type name, and the module name explicitly stored in the
> type dict as the value for key `'__module__'`.
>
> For `statically allocated type objects <static-types>`, the *tp_name*
> field should contain a dot. Everything before the last dot is made
> accessible as the `__module__` attribute, and everything after the
> last dot is made accessible as the `~definition.__name__` attribute.
>
> If no dot is present, the entire :c`~PyTypeObject.tp_name` field is
> made accessible as the `~definition.__name__` attribute, and the
> `__module__` attribute is undefined (unless explicitly set in the
> dictionary, as explained above). This means your type will be
> impossible to pickle. Additionally, it will not be listed in module
> documentations created with pydoc.
>
> This field must not be `NULL`. It is the only required field in
> :c`PyTypeObject` (other than potentially
> :c`~PyTypeObject.tp_itemsize`).
>
> **Inheritance:**
>
> This field is not inherited by subtypes.

> A pointer to the instance destructor function. This function must be
> defined unless the type guarantees that its instances will never be
> deallocated (as is the case for the singletons `None` and `Ellipsis`).
> The function signature is:
>
> ``` c
> void tp_dealloc(PyObject *self);
> ```
>
> The destructor function is called by the :c`Py_DECREF` and
> :c`Py_XDECREF` macros when the new reference count is zero. At this
> point, the instance is still in existence, but there are no references
> to it. The destructor function should free all references which the
> instance owns, free all memory buffers owned by the instance (using
> the freeing function corresponding to the allocation function used to
> allocate the buffer), and call the type's :c`~PyTypeObject.tp_free`
> function. If the type is not subtypable (doesn't have the
> `Py_TPFLAGS_BASETYPE` flag bit set), it is permissible to call the
> object deallocator directly instead of via :c`~PyTypeObject.tp_free`.
> The object deallocator should be the one used to allocate the
> instance; this is normally :c`PyObject_Del` if the instance was
> allocated using :c`PyObject_New` or :c`PyObject_VarNew`, or
> :c`PyObject_GC_Del` if the instance was allocated using
> :c`PyObject_GC_New` or :c`PyObject_GC_NewVar`.
>
> If the type supports garbage collection (has the `Py_TPFLAGS_HAVE_GC`
> flag bit set), the destructor should call :c`PyObject_GC_UnTrack`
> before clearing any member fields.
>
> ``` c
> static void foo_dealloc(foo_object *self) {
>     PyObject_GC_UnTrack(self);
>     Py_CLEAR(self->ref);
>     Py_TYPE(self)->tp_free((PyObject *)self);
> }
> ```
>
> Finally, if the type is heap allocated (`Py_TPFLAGS_HEAPTYPE`), the
> deallocator should decrement the reference count for its type object
> after calling the type deallocator. In order to avoid dangling
> pointers, the recommended way to achieve this is:
>
> ``` c
> static void foo_dealloc(foo_object *self) {
>     PyTypeObject *tp = Py_TYPE(self);
>     // free references and buffers here
>     tp->tp_free(self);
>     Py_DECREF(tp);
> }
> ```
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> An optional offset to a per-instance function that implements calling
> the object using the `vectorcall protocol <vectorcall>`, a more
> efficient alternative of the simpler :c`~PyTypeObject.tp_call`.
>
> This field is only used if the flag `Py_TPFLAGS_HAVE_VECTORCALL` is
> set. If so, this must be a positive integer containing the offset in
> the instance of a :c`vectorcallfunc` pointer.
>
> The *vectorcallfunc* pointer may be `NULL`, in which case the instance
> behaves as if `Py_TPFLAGS_HAVE_VECTORCALL` was not set: calling the
> instance falls back to :c`~PyTypeObject.tp_call`.
>
> Any class that sets `Py_TPFLAGS_HAVE_VECTORCALL` must also set
> :c`~PyTypeObject.tp_call` and make sure its behaviour is consistent
> with the *vectorcallfunc* function. This can be done by setting
> *tp_call* to :c`PyVectorcall_Call`.
>
> Warning
>
> It is not recommended for `mutable heap types <heap-types>` to
> implement the vectorcall protocol. When a user sets `__call__` in
> Python code, only *tp_call* is updated, likely making it inconsistent
> with the vectorcall function.
>
> Note
>
> The semantics of the `tp_vectorcall_offset` slot are provisional and
> expected to be finalized in Python 3.9. If you use vectorcall, plan
> for updating your code for Python 3.9.
>
> 3.8
>
> Before version 3.8, this slot was named `tp_print`. In Python 2.x, it
> was used for printing to a file. In Python 3.0 to 3.7, it was unused.
>
> **Inheritance:**
>
> This field is always inherited. However, the
> `Py_TPFLAGS_HAVE_VECTORCALL` flag is not always inherited. If it's
> not, then the subclass won't use `vectorcall <vectorcall>`, except
> when :c`PyVectorcall_Call` is explicitly called. This is in particular
> the case for types without the `Py_TPFLAGS_IMMUTABLETYPE` flag set
> (including subclasses defined in Python).

> An optional pointer to the get-attribute-string function.
>
> This field is deprecated. When it is defined, it should point to a
> function that acts the same as the :c`~PyTypeObject.tp_getattro`
> function, but taking a C string instead of a Python string object to
> give the attribute name.
>
> **Inheritance:**
>
> Group: `tp_getattr`, `tp_getattro`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_getattro`: a subtype inherits both
> :c`~PyTypeObject.tp_getattr` and :c`~PyTypeObject.tp_getattro` from
> its base type when the subtype's :c`~PyTypeObject.tp_getattr` and
> :c`~PyTypeObject.tp_getattro` are both `NULL`.

> An optional pointer to the function for setting and deleting
> attributes.
>
> This field is deprecated. When it is defined, it should point to a
> function that acts the same as the :c`~PyTypeObject.tp_setattro`
> function, but taking a C string instead of a Python string object to
> give the attribute name.
>
> **Inheritance:**
>
> Group: `tp_setattr`, `tp_setattro`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_setattro`: a subtype inherits both
> :c`~PyTypeObject.tp_setattr` and :c`~PyTypeObject.tp_setattro` from
> its base type when the subtype's :c`~PyTypeObject.tp_setattr` and
> :c`~PyTypeObject.tp_setattro` are both `NULL`.

> Pointer to an additional structure that contains fields relevant only
> to objects which implement `awaitable` and `asynchronous iterator`
> protocols at the C-level. See `async-structs` for details.
>
> 3.5 Formerly known as `tp_compare` and `tp_reserved`.
>
> **Inheritance:**
>
> The :c`~PyTypeObject.tp_as_async` field is not inherited, but the
> contained fields are inherited individually.

> builtin: repr
>
> An optional pointer to a function that implements the built-in
> function `repr`.
>
> The signature is the same as for :c`PyObject_Repr`:
>
> ``` c
> PyObject *tp_repr(PyObject *self);
> ```
>
> The function must return a string or a Unicode object. Ideally, this
> function should return a string that, when passed to `eval`, given a
> suitable environment, returns an object with the same value. If this
> is not feasible, it should return a string starting with `'<'` and
> ending with `'>'` from which both the type and the value of the object
> can be deduced.
>
> **Inheritance:**
>
> This field is inherited by subtypes.
>
> **Default:**
>
> When this field is not set, a string of the form `<%s object at %p>`
> is returned, where `%s` is replaced by the type name, and `%p` by the
> object's memory address.

> Pointer to an additional structure that contains fields relevant only
> to objects which implement the number protocol. These fields are
> documented in `number-structs`.
>
> **Inheritance:**
>
> The :c`~PyTypeObject.tp_as_number` field is not inherited, but the
> contained fields are inherited individually.

> Pointer to an additional structure that contains fields relevant only
> to objects which implement the sequence protocol. These fields are
> documented in `sequence-structs`.
>
> **Inheritance:**
>
> The :c`~PyTypeObject.tp_as_sequence` field is not inherited, but the
> contained fields are inherited individually.

> Pointer to an additional structure that contains fields relevant only
> to objects which implement the mapping protocol. These fields are
> documented in `mapping-structs`.
>
> **Inheritance:**
>
> The :c`~PyTypeObject.tp_as_mapping` field is not inherited, but the
> contained fields are inherited individually.

> builtin: hash
>
> An optional pointer to a function that implements the built-in
> function `hash`.
>
> The signature is the same as for :c`PyObject_Hash`:
>
> ``` c
> Py_hash_t tp_hash(PyObject *);
> ```
>
> The value `-1` should not be returned as a normal return value; when
> an error occurs during the computation of the hash value, the function
> should set an exception and return `-1`.
>
> When this field is not set (*and* `tp_richcompare` is not set), an
> attempt to take the hash of the object raises `TypeError`. This is the
> same as setting it to :c`PyObject_HashNotImplemented`.
>
> This field can be set explicitly to :c`PyObject_HashNotImplemented` to
> block inheritance of the hash method from a parent type. This is
> interpreted as the equivalent of `__hash__ = None` at the Python
> level, causing `isinstance(o, collections.Hashable)` to correctly
> return `False`. Note that the converse is also true - setting
> `__hash__ = None` on a class at the Python level will result in the
> `tp_hash` slot being set to :c`PyObject_HashNotImplemented`.
>
> **Inheritance:**
>
> Group: `tp_hash`, `tp_richcompare`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_richcompare`: a subtype inherits both of
> :c`~PyTypeObject.tp_richcompare` and :c`~PyTypeObject.tp_hash`, when
> the subtype's :c`~PyTypeObject.tp_richcompare` and
> :c`~PyTypeObject.tp_hash` are both `NULL`.

> An optional pointer to a function that implements calling the object.
> This should be `NULL` if the object is not callable. The signature is
> the same as for :c`PyObject_Call`:
>
> ``` c
> PyObject *tp_call(PyObject *self, PyObject *args, PyObject *kwargs);
> ```
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> An optional pointer to a function that implements the built-in
> operation `str`. (Note that `str` is a type now, and `str` calls the
> constructor for that type. This constructor calls :c`PyObject_Str` to
> do the actual work, and :c`PyObject_Str` will call this handler.)
>
> The signature is the same as for :c`PyObject_Str`:
>
> ``` c
> PyObject *tp_str(PyObject *self);
> ```
>
> The function must return a string or a Unicode object. It should be a
> "friendly" string representation of the object, as this is the
> representation that will be used, among other things, by the `print`
> function.
>
> **Inheritance:**
>
> This field is inherited by subtypes.
>
> **Default:**
>
> When this field is not set, :c`PyObject_Repr` is called to return a
> string representation.

> An optional pointer to the get-attribute function.
>
> The signature is the same as for :c`PyObject_GetAttr`:
>
> ``` c
> PyObject *tp_getattro(PyObject *self, PyObject *attr);
> ```
>
> It is usually convenient to set this field to
> :c`PyObject_GenericGetAttr`, which implements the normal way of
> looking for object attributes.
>
> **Inheritance:**
>
> Group: `tp_getattr`, `tp_getattro`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_getattr`: a subtype inherits both
> :c`~PyTypeObject.tp_getattr` and :c`~PyTypeObject.tp_getattro` from
> its base type when the subtype's :c`~PyTypeObject.tp_getattr` and
> :c`~PyTypeObject.tp_getattro` are both `NULL`.
>
> **Default:**
>
> :c`PyBaseObject_Type` uses :c`PyObject_GenericGetAttr`.

> An optional pointer to the function for setting and deleting
> attributes.
>
> The signature is the same as for :c`PyObject_SetAttr`:
>
> ``` c
> int tp_setattro(PyObject *self, PyObject *attr, PyObject *value);
> ```
>
> In addition, setting *value* to `NULL` to delete an attribute must be
> supported. It is usually convenient to set this field to
> :c`PyObject_GenericSetAttr`, which implements the normal way of
> setting object attributes.
>
> **Inheritance:**
>
> Group: `tp_setattr`, `tp_setattro`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_setattr`: a subtype inherits both
> :c`~PyTypeObject.tp_setattr` and :c`~PyTypeObject.tp_setattro` from
> its base type when the subtype's :c`~PyTypeObject.tp_setattr` and
> :c`~PyTypeObject.tp_setattro` are both `NULL`.
>
> **Default:**
>
> :c`PyBaseObject_Type` uses :c`PyObject_GenericSetAttr`.

> Pointer to an additional structure that contains fields relevant only
> to objects which implement the buffer interface. These fields are
> documented in `buffer-structs`.
>
> **Inheritance:**
>
> The :c`~PyTypeObject.tp_as_buffer` field is not inherited, but the
> contained fields are inherited individually.

> This field is a bit mask of various flags. Some flags indicate variant
> semantics for certain situations; others are used to indicate that
> certain fields in the type object (or in the extension structures
> referenced via :c`~PyTypeObject.tp_as_number`,
> :c`~PyTypeObject.tp_as_sequence`, :c`~PyTypeObject.tp_as_mapping`, and
> :c`~PyTypeObject.tp_as_buffer`) that were historically not always
> present are valid; if such a flag bit is clear, the type fields it
> guards must not be accessed and must be considered to have a zero or
> `NULL` value instead.
>
> **Inheritance:**
>
> Inheritance of this field is complicated. Most flag bits are inherited
> individually, i.e. if the base type has a flag bit set, the subtype
> inherits this flag bit. The flag bits that pertain to extension
> structures are strictly inherited if the extension structure is
> inherited, i.e. the base type's value of the flag bit is copied into
> the subtype together with a pointer to the extension structure. The
> `Py_TPFLAGS_HAVE_GC` flag bit is inherited together with the
> :c`~PyTypeObject.tp_traverse` and :c`~PyTypeObject.tp_clear` fields,
> i.e. if the `Py_TPFLAGS_HAVE_GC` flag bit is clear in the subtype and
> the :c`~PyTypeObject.tp_traverse` and :c`~PyTypeObject.tp_clear`
> fields in the subtype exist and have `NULL` values.
>
> **Default:**
>
> :c`PyBaseObject_Type` uses `Py_TPFLAGS_DEFAULT | Py_TPFLAGS_BASETYPE`.
>
> **Bit Masks:**
>
> The following bit masks are currently defined; these can be ORed
> together using the `|` operator to form the value of the
> :c`~PyTypeObject.tp_flags` field. The macro :c`PyType_HasFeature`
> takes a type and a flags value, *tp* and *f*, and checks whether
> `tp->tp_flags & f` is non-zero.
>
> Py_TPFLAGS_HEAPTYPE
>
> This bit is set when the type object itself is allocated on the heap,
> for example, types created dynamically using :c`PyType_FromSpec`. In
> this case, the `ob_type` field of its instances is considered a
> reference to the type, and the type object is INCREF'ed when a new
> instance is created, and DECREF'ed when an instance is destroyed (this
> does not apply to instances of subtypes; only the type referenced by
> the instance's ob_type gets INCREF'ed or DECREF'ed).
>
> **Inheritance:**
>
> ???
>
> Py_TPFLAGS_BASETYPE
>
> This bit is set when the type can be used as the base type of another
> type. If this bit is clear, the type cannot be subtyped (similar to a
> "final" class in Java).
>
> **Inheritance:**
>
> ???
>
> Py_TPFLAGS_READY
>
> This bit is set when the type object has been fully initialized by
> :c`PyType_Ready`.
>
> **Inheritance:**
>
> ???
>
> Py_TPFLAGS_READYING
>
> This bit is set while :c`PyType_Ready` is in the process of
> initializing the type object.
>
> **Inheritance:**
>
> ???
>
> Py_TPFLAGS_HAVE_GC
>
> This bit is set when the object supports garbage collection. If this
> bit is set, instances must be created using :c`PyObject_GC_New` and
> destroyed using :c`PyObject_GC_Del`. More information in section
> `supporting-cycle-detection`. This bit also implies that the
> GC-related fields :c`~PyTypeObject.tp_traverse` and
> :c`~PyTypeObject.tp_clear` are present in the type object.
>
> **Inheritance:**
>
> Group: `Py_TPFLAGS_HAVE_GC`, `tp_traverse`, `tp_clear`
>
> The `Py_TPFLAGS_HAVE_GC` flag bit is inherited together with the
> `tp_traverse` and `tp_clear` fields, i.e. if the `Py_TPFLAGS_HAVE_GC`
> flag bit is clear in the subtype and the `tp_traverse` and `tp_clear`
> fields in the subtype exist and have `NULL` values.
>
> Py_TPFLAGS_DEFAULT
>
> This is a bitmask of all the bits that pertain to the existence of
> certain fields in the type object and its extension structures.
> Currently, it includes the following bits:
> `Py_TPFLAGS_HAVE_STACKLESS_EXTENSION`.
>
> **Inheritance:**
>
> ???
>
> Py_TPFLAGS_METHOD_DESCRIPTOR
>
> This bit indicates that objects behave like unbound methods.
>
> If this flag is set for `type(meth)`, then:
>
> -   `meth.__get__(obj, cls)(*args, **kwds)` (with `obj` not None) must
>     be equivalent to `meth(obj, *args, **kwds)`.
> -   `meth.__get__(None, cls)(*args, **kwds)` must be equivalent to
>     `meth(*args, **kwds)`.
>
> This flag enables an optimization for typical method calls like
> `obj.meth()`: it avoids creating a temporary "bound method" object for
> `obj.meth`.
>
> 3.8
>
> **Inheritance:**
>
> This flag is never inherited by types without the
> `Py_TPFLAGS_IMMUTABLETYPE` flag set. For extension types, it is
> inherited whenever :c`~PyTypeObject.tp_descr_get` is inherited.
>
> Py_TPFLAGS_LONG_SUBCLASS
>
> Py_TPFLAGS_LIST_SUBCLASS
>
> Py_TPFLAGS_TUPLE_SUBCLASS
>
> Py_TPFLAGS_BYTES_SUBCLASS
>
> Py_TPFLAGS_UNICODE_SUBCLASS
>
> Py_TPFLAGS_DICT_SUBCLASS
>
> Py_TPFLAGS_BASE_EXC_SUBCLASS
>
> Py_TPFLAGS_TYPE_SUBCLASS
>
> These flags are used by functions such as :c`PyLong_Check` to quickly
> determine if a type is a subclass of a built-in type; such specific
> checks are faster than a generic check, like :c`PyObject_IsInstance`.
> Custom types that inherit from built-ins should have their
> :c`~PyTypeObject.tp_flags` set appropriately, or the code that
> interacts with such types will behave differently depending on what
> kind of check is used.
>
> Py_TPFLAGS_HAVE_FINALIZE
>
> This bit is set when the :c`~PyTypeObject.tp_finalize` slot is present
> in the type structure.
>
> 3.4
>
> 3.8 This flag isn't necessary anymore, as the interpreter assumes the
> :c`~PyTypeObject.tp_finalize` slot is always present in the type
> structure.
>
> Py_TPFLAGS_HAVE_VECTORCALL
>
> This bit is set when the class implements the
> `vectorcall protocol <vectorcall>`. See
> :c`~PyTypeObject.tp_vectorcall_offset` for details.
>
> **Inheritance:**
>
> This bit is inherited for types with the `Py_TPFLAGS_IMMUTABLETYPE`
> flag set, if :c`~PyTypeObject.tp_call` is also inherited.
>
> 3.9
>
> Py_TPFLAGS_IMMUTABLETYPE
>
> This bit is set for type objects that are immutable: type attributes
> cannot be set nor deleted.
>
> :c`PyType_Ready` automatically applies this flag to
> `static types <static-types>`.
>
> **Inheritance:**
>
> This flag is not inherited.
>
> 3.10
>
> Py_TPFLAGS_DISALLOW_INSTANTIATION
>
> Disallow creating instances of the type: set :c`~PyTypeObject.tp_new`
> to NULL and don't create the `__new__` key in the type dictionary.
>
> The flag must be set before creating the type, not after. For example,
> it must be set before :c`PyType_Ready` is called on the type.
>
> The flag is set automatically on `static types <static-types>` if
> :c`~PyTypeObject.tp_base` is NULL or `&PyBaseObject_Type` and
> :c`~PyTypeObject.tp_new` is NULL.
>
> **Inheritance:**
>
> This flag is not inherited.
>
> 3.10
>
> Py_TPFLAGS_MAPPING
>
> This bit indicates that instances of the class may match mapping
> patterns when used as the subject of a `match` block. It is
> automatically set when registering or subclassing
> `collections.abc.Mapping`, and unset when registering
> `collections.abc.Sequence`.
>
> Note
>
> `Py_TPFLAGS_MAPPING` and `Py_TPFLAGS_SEQUENCE` are mutually exclusive;
> it is an error to enable both flags simultaneously.
>
> **Inheritance:**
>
> This flag is inherited by types that do not already set
> `Py_TPFLAGS_SEQUENCE`.
>
> `634` -- Structural Pattern Matching: Specification
>
> 3.10
>
> Py_TPFLAGS_SEQUENCE
>
> This bit indicates that instances of the class may match sequence
> patterns when used as the subject of a `match` block. It is
> automatically set when registering or subclassing
> `collections.abc.Sequence`, and unset when registering
> `collections.abc.Mapping`.
>
> Note
>
> `Py_TPFLAGS_MAPPING` and `Py_TPFLAGS_SEQUENCE` are mutually exclusive;
> it is an error to enable both flags simultaneously.
>
> **Inheritance:**
>
> This flag is inherited by types that do not already set
> `Py_TPFLAGS_MAPPING`.
>
> `634` -- Structural Pattern Matching: Specification
>
> 3.10

> An optional pointer to a NUL-terminated C string giving the docstring
> for this type object. This is exposed as the `__doc__` attribute on
> the type and instances of the type.
>
> **Inheritance:**
>
> This field is *not* inherited by subtypes.

> An optional pointer to a traversal function for the garbage collector.
> This is only used if the `Py_TPFLAGS_HAVE_GC` flag bit is set. The
> signature is:
>
> ``` c
> int tp_traverse(PyObject *self, visitproc visit, void *arg);
> ```
>
> More information about Python's garbage collection scheme can be found
> in section `supporting-cycle-detection`.
>
> The :c`~PyTypeObject.tp_traverse` pointer is used by the garbage
> collector to detect reference cycles. A typical implementation of a
> :c`~PyTypeObject.tp_traverse` function simply calls :c`Py_VISIT` on
> each of the instance's members that are Python objects that the
> instance owns. For example, this is function :c`local_traverse` from
> the `_thread` extension module:
>
> ``` c
> static int
> local_traverse(localobject *self, visitproc visit, void *arg)
> {
>     Py_VISIT(self->args);
>     Py_VISIT(self->kw);
>     Py_VISIT(self->dict);
>     return 0;
> }
> ```
>
> Note that :c`Py_VISIT` is called only on those members that can
> participate in reference cycles. Although there is also a `self->key`
> member, it can only be `NULL` or a Python string and therefore cannot
> be part of a reference cycle.
>
> On the other hand, even if you know a member can never be part of a
> cycle, as a debugging aid you may want to visit it anyway just so the
> `gc` module's `~gc.get_referents` function will include it.
>
> Warning
>
> When implementing :c`~PyTypeObject.tp_traverse`, only the members that
> the instance *owns* (by having `strong references
> <strong reference>` to them) must be visited. For instance, if an
> object supports weak references via the :c`~PyTypeObject.tp_weaklist`
> slot, the pointer supporting the linked list (what *tp_weaklist*
> points to) must **not** be visited as the instance does not directly
> own the weak references to itself (the weakreference list is there to
> support the weak reference machinery, but the instance has no strong
> reference to the elements inside it, as they are allowed to be removed
> even if the instance is still alive).
>
> Note that :c`Py_VISIT` requires the *visit* and *arg* parameters to
> :c`local_traverse` to have these specific names; don't name them just
> anything.
>
> Instances of `heap-allocated types <heap-types>` hold a reference to
> their type. Their traversal function must therefore either visit
> :c`Py_TYPE(self) <Py_TYPE>`, or delegate this responsibility by
> calling `tp_traverse` of another heap-allocated type (such as a
> heap-allocated superclass). If they do not, the type object may not be
> garbage-collected.
>
> 3.9
>
> Heap-allocated types are expected to visit `Py_TYPE(self)` in
> `tp_traverse`. In earlier versions of Python, due to [bug
> 40217](https://bugs.python.org/issue40217), doing this may lead to
> crashes in subclasses.
>
> **Inheritance:**
>
> Group: `Py_TPFLAGS_HAVE_GC`, `tp_traverse`, `tp_clear`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_clear` and the `Py_TPFLAGS_HAVE_GC` flag bit: the
> flag bit, :c`~PyTypeObject.tp_traverse`, and
> :c`~PyTypeObject.tp_clear` are all inherited from the base type if
> they are all zero in the subtype.

> An optional pointer to a clear function for the garbage collector.
> This is only used if the `Py_TPFLAGS_HAVE_GC` flag bit is set. The
> signature is:
>
> ``` c
> int tp_clear(PyObject *);
> ```
>
> The :c`~PyTypeObject.tp_clear` member function is used to break
> reference cycles in cyclic garbage detected by the garbage collector.
> Taken together, all :c`~PyTypeObject.tp_clear` functions in the system
> must combine to break all reference cycles. This is subtle, and if in
> any doubt supply a :c`~PyTypeObject.tp_clear` function. For example,
> the tuple type does not implement a :c`~PyTypeObject.tp_clear`
> function, because it's possible to prove that no reference cycle can
> be composed entirely of tuples. Therefore the
> :c`~PyTypeObject.tp_clear` functions of other types must be sufficient
> to break any cycle containing a tuple. This isn't immediately obvious,
> and there's rarely a good reason to avoid implementing
> :c`~PyTypeObject.tp_clear`.
>
> Implementations of :c`~PyTypeObject.tp_clear` should drop the
> instance's references to those of its members that may be Python
> objects, and set its pointers to those members to `NULL`, as in the
> following example:
>
> ``` c
> static int
> local_clear(localobject *self)
> {
>     Py_CLEAR(self->key);
>     Py_CLEAR(self->args);
>     Py_CLEAR(self->kw);
>     Py_CLEAR(self->dict);
>     return 0;
> }
> ```
>
> The :c`Py_CLEAR` macro should be used, because clearing references is
> delicate: the reference to the contained object must not be
> decremented until after the pointer to the contained object is set to
> `NULL`. This is because decrementing the reference count may cause the
> contained object to become trash, triggering a chain of reclamation
> activity that may include invoking arbitrary Python code (due to
> finalizers, or weakref callbacks, associated with the contained
> object). If it's possible for such code to reference *self* again,
> it's important that the pointer to the contained object be `NULL` at
> that time, so that *self* knows the contained object can no longer be
> used. The :c`Py_CLEAR` macro performs the operations in a safe order.
>
> Note that :c`~PyTypeObject.tp_clear` is not *always* called before an
> instance is deallocated. For example, when reference counting is
> enough to determine that an object is no longer used, the cyclic
> garbage collector is not involved and :c`~PyTypeObject.tp_dealloc` is
> called directly.
>
> Because the goal of :c`~PyTypeObject.tp_clear` functions is to break
> reference cycles, it's not necessary to clear contained objects like
> Python strings or Python integers, which can't participate in
> reference cycles. On the other hand, it may be convenient to clear all
> contained Python objects, and write the type's
> :c`~PyTypeObject.tp_dealloc` function to invoke
> :c`~PyTypeObject.tp_clear`.
>
> More information about Python's garbage collection scheme can be found
> in section `supporting-cycle-detection`.
>
> **Inheritance:**
>
> Group: `Py_TPFLAGS_HAVE_GC`, `tp_traverse`, `tp_clear`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_traverse` and the `Py_TPFLAGS_HAVE_GC` flag bit:
> the flag bit, :c`~PyTypeObject.tp_traverse`, and
> :c`~PyTypeObject.tp_clear` are all inherited from the base type if
> they are all zero in the subtype.

> An optional pointer to the rich comparison function, whose signature
> is:
>
> ``` c
> PyObject *tp_richcompare(PyObject *self, PyObject *other, int op);
> ```
>
> The first parameter is guaranteed to be an instance of the type that
> is defined by :c`PyTypeObject`.
>
> The function should return the result of the comparison (usually
> `Py_True` or `Py_False`). If the comparison is undefined, it must
> return `Py_NotImplemented`, if another error occurred it must return
> `NULL` and set an exception condition.
>
> The following constants are defined to be used as the third argument
> for :c`~PyTypeObject.tp_richcompare` and for :c`PyObject_RichCompare`:
>
> | Constant | Comparison |
> |----------|------------|
> | `Py_LT`  | `<`        |
> | `Py_LE`  | `<=`       |
> | `Py_EQ`  | `==`       |
> | `Py_NE`  | `!=`       |
> | `Py_GT`  | `>`        |
> | `Py_GE`  | `>=`       |
>
> The following macro is defined to ease writing rich comparison
> functions:
>
> > Return `Py_True` or `Py_False` from the function, depending on the
> > result of a comparison. VAL_A and VAL_B must be orderable by C
> > comparison operators (for example, they may be C ints or floats).
> > The third argument specifies the requested operation, as for
> > :c`PyObject_RichCompare`.
> >
> > The return value's reference count is properly incremented.
> >
> > On error, sets an exception and returns `NULL` from the function.
> >
> > 3.7
>
> **Inheritance:**
>
> Group: `tp_hash`, `tp_richcompare`
>
> This field is inherited by subtypes together with
> :c`~PyTypeObject.tp_hash`: a subtype inherits
> :c`~PyTypeObject.tp_richcompare` and :c`~PyTypeObject.tp_hash` when
> the subtype's :c`~PyTypeObject.tp_richcompare` and
> :c`~PyTypeObject.tp_hash` are both `NULL`.
>
> **Default:**
>
> :c`PyBaseObject_Type` provides a `tp_richcompare` implementation,
> which may be inherited. However, if only `tp_hash` is defined, not
> even the inherited function is used and instances of the type will not
> be able to participate in any comparisons.

> If the instances of this type are weakly referenceable, this field is
> greater than zero and contains the offset in the instance structure of
> the weak reference list head (ignoring the GC header, if present);
> this offset is used by :c`PyObject_ClearWeakRefs` and the
> :c`PyWeakref_\*` functions. The instance structure needs to include a
> field of type :c`PyObject*` which is initialized to `NULL`.
>
> Do not confuse this field with :c`~PyTypeObject.tp_weaklist`; that is
> the list head for weak references to the type object itself.
>
> **Inheritance:**
>
> This field is inherited by subtypes, but see the rules listed below. A
> subtype may override this offset; this means that the subtype uses a
> different weak reference list head than the base type. Since the list
> head is always found via :c`~PyTypeObject.tp_weaklistoffset`, this
> should not be a problem.
>
> When a type defined by a class statement has no `~object.__slots__`
> declaration, and none of its base types are weakly referenceable, the
> type is made weakly referenceable by adding a weak reference list head
> slot to the instance layout and setting the
> :c`~PyTypeObject.tp_weaklistoffset` of that slot's offset.
>
> When a type's `__slots__` declaration contains a slot named
> `__weakref__`, that slot becomes the weak reference list head for
> instances of the type, and the slot's offset is stored in the type's
> :c`~PyTypeObject.tp_weaklistoffset`.
>
> When a type's `__slots__` declaration does not contain a slot named
> `__weakref__`, the type inherits its
> :c`~PyTypeObject.tp_weaklistoffset` from its base type.

> An optional pointer to a function that returns an `iterator` for the
> object. Its presence normally signals that the instances of this type
> are `iterable` (although sequences may be iterable without this
> function).
>
> This function has the same signature as :c`PyObject_GetIter`:
>
> ``` c
> PyObject *tp_iter(PyObject *self);
> ```
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> An optional pointer to a function that returns the next item in an
> `iterator`. The signature is:
>
> ``` c
> PyObject *tp_iternext(PyObject *self);
> ```
>
> When the iterator is exhausted, it must return `NULL`; a
> `StopIteration` exception may or may not be set. When another error
> occurs, it must return `NULL` too. Its presence signals that the
> instances of this type are iterators.
>
> Iterator types should also define the :c`~PyTypeObject.tp_iter`
> function, and that function should return the iterator instance itself
> (not a new iterator instance).
>
> This function has the same signature as :c`PyIter_Next`.
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> An optional pointer to a static `NULL`-terminated array of
> :c`PyMethodDef` structures, declaring regular methods of this type.
>
> For each entry in the array, an entry is added to the type's
> dictionary (see :c`~PyTypeObject.tp_dict` below) containing a method
> descriptor.
>
> **Inheritance:**
>
> This field is not inherited by subtypes (methods are inherited through
> a different mechanism).

> An optional pointer to a static `NULL`-terminated array of
> :c`PyMemberDef` structures, declaring regular data members (fields or
> slots) of instances of this type.
>
> For each entry in the array, an entry is added to the type's
> dictionary (see :c`~PyTypeObject.tp_dict` below) containing a member
> descriptor.
>
> **Inheritance:**
>
> This field is not inherited by subtypes (members are inherited through
> a different mechanism).

> An optional pointer to a static `NULL`-terminated array of
> :c`PyGetSetDef` structures, declaring computed attributes of instances
> of this type.
>
> For each entry in the array, an entry is added to the type's
> dictionary (see :c`~PyTypeObject.tp_dict` below) containing a getset
> descriptor.
>
> **Inheritance:**
>
> This field is not inherited by subtypes (computed attributes are
> inherited through a different mechanism).

> An optional pointer to a base type from which type properties are
> inherited. At this level, only single inheritance is supported;
> multiple inheritance require dynamically creating a type object by
> calling the metatype.
>
> Note
>
> Slot initialization is subject to the rules of initializing globals.
> C99 requires the initializers to be "address constants". Function
> designators like :c`PyType_GenericNew`, with implicit conversion to a
> pointer, are valid C99 address constants.
>
> However, the unary '&' operator applied to a non-static variable like
> :c`PyBaseObject_Type` is not required to produce an address constant.
> Compilers may support this (gcc does), MSVC does not. Both compilers
> are strictly standard conforming in this particular behavior.
>
> Consequently, :c`~PyTypeObject.tp_base` should be set in the extension
> module's init function.
>
> **Inheritance:**
>
> This field is not inherited by subtypes (obviously).
>
> **Default:**
>
> This field defaults to `&PyBaseObject_Type` (which to Python
> programmers is known as the type `object`).

> The type's dictionary is stored here by :c`PyType_Ready`.
>
> This field should normally be initialized to `NULL` before
> PyType_Ready is called; it may also be initialized to a dictionary
> containing initial attributes for the type. Once :c`PyType_Ready` has
> initialized the type, extra attributes for the type may be added to
> this dictionary only if they don't correspond to overloaded operations
> (like `__add__`).
>
> **Inheritance:**
>
> This field is not inherited by subtypes (though the attributes defined
> in here are inherited through a different mechanism).
>
> **Default:**
>
> If this field is `NULL`, :c`PyType_Ready` will assign a new dictionary
> to it.
>
> Warning
>
> It is not safe to use :c`PyDict_SetItem` on or otherwise modify
> :c`~PyTypeObject.tp_dict` with the dictionary C-API.

> An optional pointer to a "descriptor get" function.
>
> The function signature is:
>
> ``` c
> PyObject * tp_descr_get(PyObject *self, PyObject *obj, PyObject *type);
> ```
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> An optional pointer to a function for setting and deleting a
> descriptor's value.
>
> The function signature is:
>
> ``` c
> int tp_descr_set(PyObject *self, PyObject *obj, PyObject *value);
> ```
>
> The *value* argument is set to `NULL` to delete the value.
>
> **Inheritance:**
>
> This field is inherited by subtypes.

> If the instances of this type have a dictionary containing instance
> variables, this field is non-zero and contains the offset in the
> instances of the type of the instance variable dictionary; this offset
> is used by :c`PyObject_GenericGetAttr`.
>
> Do not confuse this field with :c`~PyTypeObject.tp_dict`; that is the
> dictionary for attributes of the type object itself.
>
> If the value of this field is greater than zero, it specifies the
> offset from the start of the instance structure. If the value is less
> than zero, it specifies the offset from the *end* of the instance
> structure. A negative offset is more expensive to use, and should only
> be used when the instance structure contains a variable-length part.
> This is used for example to add an instance variable dictionary to
> subtypes of `str` or `tuple`. Note that the
> :c`~PyTypeObject.tp_basicsize` field should account for the dictionary
> added to the end in that case, even though the dictionary is not
> included in the basic object layout. On a system with a pointer size
> of 4 bytes, :c`~PyTypeObject.tp_dictoffset` should be set to `-4` to
> indicate that the dictionary is at the very end of the structure.
>
> The real dictionary offset in an instance can be computed from a
> negative :c`~PyTypeObject.tp_dictoffset` as follows:
>
> ``` c
> dictoffset = tp_basicsize + abs(ob_size)*tp_itemsize + tp_dictoffset
> if dictoffset is not aligned on sizeof(void*):
>     round up to sizeof(void*)
> ```
>
> where :c`~PyTypeObject.tp_basicsize`, :c`~PyTypeObject.tp_itemsize`
> and :c`~PyTypeObject.tp_dictoffset` are taken from the type object,
> and `ob_size` is taken from the instance. The absolute value is taken
> because ints use the sign of `ob_size` to store the sign of the
> number. (There's never a need to do this calculation yourself; it is
> done for you by :c`_PyObject_GetDictPtr`.)
>
> **Inheritance:**
>
> This field is inherited by subtypes, but see the rules listed below. A
> subtype may override this offset; this means that the subtype
> instances store the dictionary at a difference offset than the base
> type. Since the dictionary is always found via
> :c`~PyTypeObject.tp_dictoffset`, this should not be a problem.
>
> When a type defined by a class statement has no `~object.__slots__`
> declaration, and none of its base types has an instance variable
> dictionary, a dictionary slot is added to the instance layout and the
> :c`~PyTypeObject.tp_dictoffset` is set to that slot's offset.
>
> When a type defined by a class statement has a `__slots__`
> declaration, the type inherits its :c`~PyTypeObject.tp_dictoffset`
> from its base type.
>
> (Adding a slot named `~object.__dict__` to the `__slots__` declaration
> does not have the expected effect, it just causes confusion. Maybe
> this should be added as a feature just like `__weakref__` though.)
>
> **Default:**
>
> This slot has no default. For `static types <static-types>`, if the
> field is `NULL` then no `__dict__` gets created for instances.

> An optional pointer to an instance initialization function.
>
> This function corresponds to the `__init__` method of classes. Like
> `__init__`, it is possible to create an instance without calling
> `__init__`, and it is possible to reinitialize an instance by calling
> its `__init__` method again.
>
> The function signature is:
>
> ``` c
> int tp_init(PyObject *self, PyObject *args, PyObject *kwds);
> ```
>
> The self argument is the instance to be initialized; the *args* and
> *kwds* arguments represent positional and keyword arguments of the
> call to `__init__`.
>
> The :c`~PyTypeObject.tp_init` function, if not `NULL`, is called when
> an instance is created normally by calling its type, after the type's
> :c`~PyTypeObject.tp_new` function has returned an instance of the
> type. If the :c`~PyTypeObject.tp_new` function returns an instance of
> some other type that is not a subtype of the original type, no
> :c`~PyTypeObject.tp_init` function is called; if
> :c`~PyTypeObject.tp_new` returns an instance of a subtype of the
> original type, the subtype's :c`~PyTypeObject.tp_init` is called.
>
> Returns `0` on success, `-1` and sets an exception on error.
>
> **Inheritance:**
>
> This field is inherited by subtypes.
>
> **Default:**
>
> For `static types <static-types>` this field does not have a default.

> An optional pointer to an instance allocation function.
>
> The function signature is:
>
> ``` c
> PyObject *tp_alloc(PyTypeObject *self, Py_ssize_t nitems);
> ```
>
> **Inheritance:**
>
> This field is inherited by static subtypes, but not by dynamic
> subtypes (subtypes created by a class statement).
>
> **Default:**
>
> For dynamic subtypes, this field is always set to
> :c`PyType_GenericAlloc`, to force a standard heap allocation strategy.
>
> For static subtypes, :c`PyBaseObject_Type` uses
> :c`PyType_GenericAlloc`. That is the recommended value for all
> statically defined types.

> An optional pointer to an instance creation function.
>
> The function signature is:
>
> ``` c
> PyObject *tp_new(PyTypeObject *subtype, PyObject *args, PyObject *kwds);
> ```
>
> The *subtype* argument is the type of the object being created; the
> *args* and *kwds* arguments represent positional and keyword arguments
> of the call to the type. Note that *subtype* doesn't have to equal the
> type whose :c`~PyTypeObject.tp_new` function is called; it may be a
> subtype of that type (but not an unrelated type).
>
> The :c`~PyTypeObject.tp_new` function should call
> `subtype->tp_alloc(subtype, nitems)` to allocate space for the object,
> and then do only as much further initialization as is absolutely
> necessary. Initialization that can safely be ignored or repeated
> should be placed in the :c`~PyTypeObject.tp_init` handler. A good rule
> of thumb is that for immutable types, all initialization should take
> place in :c`~PyTypeObject.tp_new`, while for mutable types, most
> initialization should be deferred to :c`~PyTypeObject.tp_init`.
>
> Set the `Py_TPFLAGS_DISALLOW_INSTANTIATION` flag to disallow creating
> instances of the type in Python.
>
> **Inheritance:**
>
> This field is inherited by subtypes, except it is not inherited by
> `static types <static-types>` whose :c`~PyTypeObject.tp_base` is
> `NULL` or `&PyBaseObject_Type`.
>
> **Default:**
>
> For `static types <static-types>` this field has no default. This
> means if the slot is defined as `NULL`, the type cannot be called to
> create new instances; presumably there is some other way to create
> instances, like a factory function.

> An optional pointer to an instance deallocation function. Its
> signature is:
>
> ``` c
> void tp_free(void *self);
> ```
>
> An initializer that is compatible with this signature is
> :c`PyObject_Free`.
>
> **Inheritance:**
>
> This field is inherited by static subtypes, but not by dynamic
> subtypes (subtypes created by a class statement)
>
> **Default:**
>
> In dynamic subtypes, this field is set to a deallocator suitable to
> match :c`PyType_GenericAlloc` and the value of the
> `Py_TPFLAGS_HAVE_GC` flag bit.
>
> For static subtypes, :c`PyBaseObject_Type` uses PyObject_Del.

> An optional pointer to a function called by the garbage collector.
>
> The garbage collector needs to know whether a particular object is
> collectible or not. Normally, it is sufficient to look at the object's
> type's :c`~PyTypeObject.tp_flags` field, and check the
> `Py_TPFLAGS_HAVE_GC` flag bit. But some types have a mixture of
> statically and dynamically allocated instances, and the statically
> allocated instances are not collectible. Such types should define this
> function; it should return `1` for a collectible instance, and `0` for
> a non-collectible instance. The signature is:
>
> ``` c
> int tp_is_gc(PyObject *self);
> ```
>
> (The only example of this are types themselves. The metatype,
> :c`PyType_Type`, defines this function to distinguish between
> statically and `dynamically allocated types <heap-types>`.)
>
> **Inheritance:**
>
> This field is inherited by subtypes.
>
> **Default:**
>
> This slot has no default. If this field is `NULL`,
> `Py_TPFLAGS_HAVE_GC` is used as the functional equivalent.

> Tuple of base types.
>
> This is set for types created by a class statement. It should be
> `NULL` for statically defined types.
>
> **Inheritance:**
>
> This field is not inherited.

> Tuple containing the expanded set of base types, starting with the
> type itself and ending with `object`, in Method Resolution Order.
>
> **Inheritance:**
>
> This field is not inherited; it is calculated fresh by
> :c`PyType_Ready`.

> Unused. Internal use only.
>
> **Inheritance:**
>
> This field is not inherited.

> List of weak references to subclasses. Internal use only.
>
> **Inheritance:**
>
> This field is not inherited.

> Weak reference list head, for weak references to this type object. Not
> inherited. Internal use only.
>
> **Inheritance:**
>
> This field is not inherited.

> This field is deprecated. Use :c`~PyTypeObject.tp_finalize` instead.

> Used to index into the method cache. Internal use only.
>
> **Inheritance:**
>
> This field is not inherited.

> An optional pointer to an instance finalization function. Its
> signature is:
>
> ``` c
> void tp_finalize(PyObject *self);
> ```
>
> If :c`~PyTypeObject.tp_finalize` is set, the interpreter calls it once
> when finalizing an instance. It is called either from the garbage
> collector (if the instance is part of an isolated reference cycle) or
> just before the object is deallocated. Either way, it is guaranteed to
> be called before attempting to break reference cycles, ensuring that
> it finds the object in a sane state.
>
> :c`~PyTypeObject.tp_finalize` should not mutate the current exception
> status; therefore, a recommended way to write a non-trivial finalizer
> is:
>
> ``` c
> static void
> local_finalize(PyObject *self)
> {
>     PyObject *error_type, *error_value, *error_traceback;
>
>     /* Save the current exception, if any. */
>     PyErr_Fetch(&error_type, &error_value, &error_traceback);
>
>     /* ... */
>
>     /* Restore the saved exception. */
>     PyErr_Restore(error_type, error_value, error_traceback);
> }
> ```
>
> For this field to be taken into account (even through inheritance),
> you must also set the `Py_TPFLAGS_HAVE_FINALIZE` flags bit.
>
> Also, note that, in a garbage collected Python,
> :c`~PyTypeObject.tp_dealloc` may be called from any Python thread, not
> just the thread which created the object (if the object becomes part
> of a refcount cycle, that cycle might be collected by a garbage
> collection on any thread). This is not a problem for Python API calls,
> since the thread on which tp_dealloc is called will own the Global
> Interpreter Lock (GIL). However, if the object being destroyed in turn
> destroys objects from some other C or C++ library, care should be
> taken to ensure that destroying those objects on the thread which
> called tp_dealloc will not violate any assumptions of the library.
>
> **Inheritance:**
>
> This field is inherited by subtypes.
>
> 3.4
>
> "Safe object finalization" (`442`)

> Vectorcall function to use for calls of this type object. In other
> words, it is used to implement `vectorcall <vectorcall>` for
> `type.__call__`. If `tp_vectorcall` is `NULL`, the default call
> implementation using `__new__` and `__init__` is used.
>
> **Inheritance:**
>
> This field is never inherited.
>
> 3.9 (the field exists since 3.8 but it's only used since 3.9)

## Static Types

Traditionally, types defined in C code are *static*, that is, a static
:c`PyTypeObject` structure is defined directly in code and initialized
using :c`PyType_Ready`.

This results in types that are limited relative to types defined in
Python:

-   Static types are limited to one base, i.e. they cannot use multiple
    inheritance.
-   Static type objects (but not necessarily their instances) are
    immutable. It is not possible to add or modify the type object's
    attributes from Python.
-   Static type objects are shared across
    `sub-interpreters <sub-interpreter-support>`, so they should not
    include any subinterpreter-specific state.

Also, since :c`PyTypeObject` is not part of the `stable ABI <stable>`,
any extension modules using static types must be compiled for a specific
Python minor version.

## Heap Types

An alternative to `static types <static-types>` is *heap-allocated
types*, or *heap types* for short, which correspond closely to classes
created by Python's `class` statement. Heap types have the
`Py_TPFLAGS_HEAPTYPE` flag set.

This is done by filling a :c`PyType_Spec` structure and calling
:c`PyType_FromSpec`, :c`PyType_FromSpecWithBases`,
:c`PyType_FromModuleAndSpec`, or :c`PyType_FromMetaclass`.

# Number Object Structures

Amaury Forgeot d'Arc

> This structure holds pointers to the functions which an object uses to
> implement the number protocol. Each function is used by the function
> of similar name documented in the `number` section.
>
> Here is the structure definition:
>
> ``` c
> typedef struct {
>      binaryfunc nb_add;
>      binaryfunc nb_subtract;
>      binaryfunc nb_multiply;
>      binaryfunc nb_remainder;
>      binaryfunc nb_divmod;
>      ternaryfunc nb_power;
>      unaryfunc nb_negative;
>      unaryfunc nb_positive;
>      unaryfunc nb_absolute;
>      inquiry nb_bool;
>      unaryfunc nb_invert;
>      binaryfunc nb_lshift;
>      binaryfunc nb_rshift;
>      binaryfunc nb_and;
>      binaryfunc nb_xor;
>      binaryfunc nb_or;
>      unaryfunc nb_int;
>      void *nb_reserved;
>      unaryfunc nb_float;
>
>      binaryfunc nb_inplace_add;
>      binaryfunc nb_inplace_subtract;
>      binaryfunc nb_inplace_multiply;
>      binaryfunc nb_inplace_remainder;
>      ternaryfunc nb_inplace_power;
>      binaryfunc nb_inplace_lshift;
>      binaryfunc nb_inplace_rshift;
>      binaryfunc nb_inplace_and;
>      binaryfunc nb_inplace_xor;
>      binaryfunc nb_inplace_or;
>
>      binaryfunc nb_floor_divide;
>      binaryfunc nb_true_divide;
>      binaryfunc nb_inplace_floor_divide;
>      binaryfunc nb_inplace_true_divide;
>
>      unaryfunc nb_index;
>
>      binaryfunc nb_matrix_multiply;
>      binaryfunc nb_inplace_matrix_multiply;
> } PyNumberMethods;
> ```
>
> Note
>
> Binary and ternary functions must check the type of all their
> operands, and implement the necessary conversions (at least one of the
> operands is an instance of the defined type). If the operation is not
> defined for the given operands, binary and ternary functions must
> return `Py_NotImplemented`, if another error occurred they must return
> `NULL` and set an exception.
>
> Note
>
> The :c`nb_reserved` field should always be `NULL`. It was previously
> called :c`nb_long`, and was renamed in Python 3.0.1.

# Mapping Object Structures

Amaury Forgeot d'Arc

> This structure holds pointers to the functions which an object uses to
> implement the mapping protocol. It has three members:

> This function is used by :c`PyMapping_Size` and :c`PyObject_Size`, and
> has the same signature. This slot may be set to `NULL` if the object
> has no defined length.

> This function is used by :c`PyObject_GetItem` and
> :c`PySequence_GetSlice`, and has the same signature as
> :c`!PyObject_GetItem`. This slot must be filled for the
> :c`PyMapping_Check` function to return `1`, it can be `NULL`
> otherwise.

> This function is used by :c`PyObject_SetItem`, :c`PyObject_DelItem`,
> :c`PyObject_SetSlice` and :c`PyObject_DelSlice`. It has the same
> signature as :c`!PyObject_SetItem`, but *v* can also be set to `NULL`
> to delete an item. If this slot is `NULL`, the object does not support
> item assignment and deletion.

# Sequence Object Structures

Amaury Forgeot d'Arc

> This structure holds pointers to the functions which an object uses to
> implement the sequence protocol.

> This function is used by :c`PySequence_Size` and :c`PyObject_Size`,
> and has the same signature. It is also used for handling negative
> indices via the :c`~PySequenceMethods.sq_item` and the
> :c`~PySequenceMethods.sq_ass_item` slots.

> This function is used by :c`PySequence_Concat` and has the same
> signature. It is also used by the `+` operator, after trying the
> numeric addition via the :c`~PyNumberMethods.nb_add` slot.

> This function is used by :c`PySequence_Repeat` and has the same
> signature. It is also used by the `*` operator, after trying numeric
> multiplication via the :c`~PyNumberMethods.nb_multiply` slot.

> This function is used by :c`PySequence_GetItem` and has the same
> signature. It is also used by :c`PyObject_GetItem`, after trying the
> subscription via the :c`~PyMappingMethods.mp_subscript` slot. This
> slot must be filled for the :c`PySequence_Check` function to return
> `1`, it can be `NULL` otherwise.
>
> Negative indexes are handled as follows: if the `sq_length` slot is
> filled, it is called and the sequence length is used to compute a
> positive index which is passed to `sq_item`. If `sq_length` is `NULL`,
> the index is passed as is to the function.

> This function is used by :c`PySequence_SetItem` and has the same
> signature. It is also used by :c`PyObject_SetItem` and
> :c`PyObject_DelItem`, after trying the item assignment and deletion
> via the :c`~PyMappingMethods.mp_ass_subscript` slot. This slot may be
> left to `NULL` if the object does not support item assignment and
> deletion.

> This function may be used by :c`PySequence_Contains` and has the same
> signature. This slot may be left to `NULL`, in this case
> :c`!PySequence_Contains` simply traverses the sequence until it finds
> a match.

> This function is used by :c`PySequence_InPlaceConcat` and has the same
> signature. It should modify its first operand, and return it. This
> slot may be left to `NULL`, in this case :c`!PySequence_InPlaceConcat`
> will fall back to :c`PySequence_Concat`. It is also used by the
> augmented assignment `+=`, after trying numeric in-place addition via
> the :c`~PyNumberMethods.nb_inplace_add` slot.

> This function is used by :c`PySequence_InPlaceRepeat` and has the same
> signature. It should modify its first operand, and return it. This
> slot may be left to `NULL`, in this case :c`!PySequence_InPlaceRepeat`
> will fall back to :c`PySequence_Repeat`. It is also used by the
> augmented assignment `*=`, after trying numeric in-place
> multiplication via the :c`~PyNumberMethods.nb_inplace_multiply` slot.

# Buffer Object Structures

Greg J. Stein \<<greg@lyra.org>\>

Benjamin Peterson

Stefan Krah

> This structure holds pointers to the functions required by the
> `Buffer protocol <bufferobjects>`. The protocol defines how an
> exporter object can expose its internal data to consumer objects.

> The signature of this function is:
>
> ``` c
> int (PyObject *exporter, Py_buffer *view, int flags);
> ```
>
> Handle a request to *exporter* to fill in *view* as specified by
> *flags*. Except for point (3), an implementation of this function MUST
> take these steps:
>
> 1.  Check if the request can be met. If not, raise
>     :c`PyExc_BufferError`, set :c`view->obj` to `NULL` and return
>     `-1`.
> 2.  Fill in the requested fields.
> 3.  Increment an internal counter for the number of exports.
> 4.  Set :c`view->obj` to *exporter* and increment :c`view->obj`.
> 5.  Return `0`.
>
> If *exporter* is part of a chain or tree of buffer providers, two main
> schemes can be used:
>
> -   Re-export: Each member of the tree acts as the exporting object
>     and sets :c`view->obj` to a new reference to itself.
> -   Redirect: The buffer request is redirected to the root object of
>     the tree. Here, :c`view->obj` will be a new reference to the root
>     object.
>
> The individual fields of *view* are described in section
> `Buffer structure <buffer-structure>`, the rules how an exporter must
> react to specific requests are in section
> `Buffer request types <buffer-request-types>`.
>
> All memory pointed to in the :c`Py_buffer` structure belongs to the
> exporter and must remain valid until there are no consumers left.
> :c`~Py_buffer.format`, :c`~Py_buffer.shape`, :c`~Py_buffer.strides`,
> :c`~Py_buffer.suboffsets` and :c`~Py_buffer.internal` are read-only
> for the consumer.
>
> :c`PyBuffer_FillInfo` provides an easy way of exposing a simple bytes
> buffer while dealing correctly with all request types.
>
> :c`PyObject_GetBuffer` is the interface for the consumer that wraps
> this function.

> The signature of this function is:
>
> ``` c
> void (PyObject *exporter, Py_buffer *view);
> ```
>
> Handle a request to release the resources of the buffer. If no
> resources need to be released, :c`PyBufferProcs.bf_releasebuffer` may
> be `NULL`. Otherwise, a standard implementation of this function will
> take these optional steps:
>
> 1.  Decrement an internal counter for the number of exports.
> 2.  If the counter is `0`, free all memory associated with *view*.
>
> The exporter MUST use the :c`~Py_buffer.internal` field to keep track
> of buffer-specific resources. This field is guaranteed to remain
> constant, while a consumer MAY pass a copy of the original buffer as
> the *view* argument.
>
> This function MUST NOT decrement :c`view->obj`, since that is done
> automatically in :c`PyBuffer_Release` (this scheme is useful for
> breaking reference cycles).
>
> :c`PyBuffer_Release` is the interface for the consumer that wraps this
> function.

# Async Object Structures

Yury Selivanov \<<yselivanov@sprymix.com>\>

3.5

> This structure holds pointers to the functions required to implement
> `awaitable` and `asynchronous iterator` objects.
>
> Here is the structure definition:
>
> ``` c
> typedef struct {
>     unaryfunc am_await;
>     unaryfunc am_aiter;
>     unaryfunc am_anext;
>     sendfunc am_send;
> } PyAsyncMethods;
> ```

> The signature of this function is:
>
> ``` c
> PyObject *am_await(PyObject *self);
> ```
>
> The returned object must be an `iterator`, i.e. :c`PyIter_Check` must
> return `1` for it.
>
> This slot may be set to `NULL` if an object is not an `awaitable`.

> The signature of this function is:
>
> ``` c
> PyObject *am_aiter(PyObject *self);
> ```
>
> Must return an `asynchronous iterator` object. See `__anext__` for
> details.
>
> This slot may be set to `NULL` if an object does not implement
> asynchronous iteration protocol.

> The signature of this function is:
>
> ``` c
> PyObject *am_anext(PyObject *self);
> ```
>
> Must return an `awaitable` object. See `__anext__` for details. This
> slot may be set to `NULL`.

> The signature of this function is:
>
> ``` c
> PySendResult am_send(PyObject *self, PyObject *arg, PyObject **result);
> ```
>
> See :c`PyIter_Send` for details. This slot may be set to `NULL`.
>
> 3.10

# Slot Type typedefs

> The purpose of this function is to separate memory allocation from
> memory initialization. It should return a pointer to a block of memory
> of adequate length for the instance, suitably aligned, and initialized
> to zeros, but with `ob_refcnt` set to `1` and `ob_type` set to the
> type argument. If the type's :c`~PyTypeObject.tp_itemsize` is
> non-zero, the object's `ob_size` field should be initialized to
> *nitems* and the length of the allocated memory block should be
> `tp_basicsize + nitems*tp_itemsize`, rounded up to a multiple of
> `sizeof(void*)`; otherwise, *nitems* is not used and the length of the
> block should be :c`~PyTypeObject.tp_basicsize`.
>
> This function should not do any other instance initialization, not
> even to allocate additional memory; that should be done by
> :c`~PyTypeObject.tp_new`.

> See :c`~PyTypeObject.tp_free`.

> See :c`~PyTypeObject.tp_new`.

> See :c`~PyTypeObject.tp_init`.

> See :c`~PyTypeObject.tp_repr`.

> Return the value of the named attribute for the object.

> Set the value of the named attribute for the object. The value
> argument is set to `NULL` to delete the attribute.

> Return the value of the named attribute for the object.
>
> See :c`~PyTypeObject.tp_getattro`.

> Set the value of the named attribute for the object. The value
> argument is set to `NULL` to delete the attribute.
>
> See :c`~PyTypeObject.tp_setattro`.

> See :c`~PyTypeObject.tp_descrget`.

> See :c`~PyTypeObject.tp_descrset`.

> See :c`~PyTypeObject.tp_hash`.

> See :c`~PyTypeObject.tp_richcompare`.

> See :c`~PyTypeObject.tp_iter`.

> See :c`~PyTypeObject.tp_iternext`.

> See :c`~PyAsyncMethods.am_send`.

# Examples

The following are simple examples of Python type definitions. They
include common usage you may encounter. Some demonstrate tricky corner
cases. For more examples, practical info, and a tutorial, see
`defining-new-types` and `new-types-topics`.

A basic `static type <static-types>`:

``` c
typedef struct {
    PyObject_HEAD
    const char *data;
} MyObject;

static PyTypeObject MyObject_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    .tp_name = "mymod.MyObject",
    .tp_basicsize = sizeof(MyObject),
    .tp_doc = PyDoc_STR("My objects"),
    .tp_new = myobj_new,
    .tp_dealloc = (destructor)myobj_dealloc,
    .tp_repr = (reprfunc)myobj_repr,
};
```

You may also find older code (especially in the CPython code base) with
a more verbose initializer:

``` c
static PyTypeObject MyObject_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    "mymod.MyObject",               /* tp_name */
    sizeof(MyObject),               /* tp_basicsize */
    0,                              /* tp_itemsize */
    (destructor)myobj_dealloc,      /* tp_dealloc */
    0,                              /* tp_vectorcall_offset */
    0,                              /* tp_getattr */
    0,                              /* tp_setattr */
    0,                              /* tp_as_async */
    (reprfunc)myobj_repr,           /* tp_repr */
    0,                              /* tp_as_number */
    0,                              /* tp_as_sequence */
    0,                              /* tp_as_mapping */
    0,                              /* tp_hash */
    0,                              /* tp_call */
    0,                              /* tp_str */
    0,                              /* tp_getattro */
    0,                              /* tp_setattro */
    0,                              /* tp_as_buffer */
    0,                              /* tp_flags */
    PyDoc_STR("My objects"),        /* tp_doc */
    0,                              /* tp_traverse */
    0,                              /* tp_clear */
    0,                              /* tp_richcompare */
    0,                              /* tp_weaklistoffset */
    0,                              /* tp_iter */
    0,                              /* tp_iternext */
    0,                              /* tp_methods */
    0,                              /* tp_members */
    0,                              /* tp_getset */
    0,                              /* tp_base */
    0,                              /* tp_dict */
    0,                              /* tp_descr_get */
    0,                              /* tp_descr_set */
    0,                              /* tp_dictoffset */
    0,                              /* tp_init */
    0,                              /* tp_alloc */
    myobj_new,                      /* tp_new */
};
```

A type that supports weakrefs, instance dicts, and hashing:

``` c
typedef struct {
    PyObject_HEAD
    const char *data;
    PyObject *inst_dict;
    PyObject *weakreflist;
} MyObject;

static PyTypeObject MyObject_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    .tp_name = "mymod.MyObject",
    .tp_basicsize = sizeof(MyObject),
    .tp_doc = PyDoc_STR("My objects"),
    .tp_weaklistoffset = offsetof(MyObject, weakreflist),
    .tp_dictoffset = offsetof(MyObject, inst_dict),
    .tp_flags = Py_TPFLAGS_DEFAULT | Py_TPFLAGS_BASETYPE | Py_TPFLAGS_HAVE_GC,
    .tp_new = myobj_new,
    .tp_traverse = (traverseproc)myobj_traverse,
    .tp_clear = (inquiry)myobj_clear,
    .tp_alloc = PyType_GenericNew,
    .tp_dealloc = (destructor)myobj_dealloc,
    .tp_repr = (reprfunc)myobj_repr,
    .tp_hash = (hashfunc)myobj_hash,
    .tp_richcompare = PyBaseObject_Type.tp_richcompare,
};
```

A str subclass that cannot be subclassed and cannot be called to create
instances (e.g. uses a separate factory func) using
:c`Py_TPFLAGS_DISALLOW_INSTANTIATION` flag:

``` c
typedef struct {
    PyUnicodeObject raw;
    char *extra;
} MyStr;

static PyTypeObject MyStr_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    .tp_name = "mymod.MyStr",
    .tp_basicsize = sizeof(MyStr),
    .tp_base = NULL,  // set to &PyUnicode_Type in module init
    .tp_doc = PyDoc_STR("my custom str"),
    .tp_flags = Py_TPFLAGS_DEFAULT | Py_TPFLAGS_DISALLOW_INSTANTIATION,
    .tp_repr = (reprfunc)myobj_repr,
};
```

The simplest `static type <static-types>` with fixed-length instances:

``` c
typedef struct {
    PyObject_HEAD
} MyObject;

static PyTypeObject MyObject_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    .tp_name = "mymod.MyObject",
};
```

The simplest `static type <static-types>` with variable-length
instances:

``` c
typedef struct {
    PyObject_VAR_HEAD
    const char *data[1];
} MyObject;

static PyTypeObject MyObject_Type = {
    PyVarObject_HEAD_INIT(NULL, 0)
    .tp_name = "mymod.MyObject",
    .tp_basicsize = sizeof(MyObject) - sizeof(char *),
    .tp_itemsize = sizeof(char *),
};
```